In [1]:
from typing import List
import fitz  
from fitz.fitz import Point, Page, Rect
from pprint import pprint
from itertools import groupby
import json

# optional
#import pdfkit


In [2]:
print(fitz.__doc__)


PyMuPDF 1.19.5: Python bindings for the MuPDF 1.19.0 library.
Version date: 2022-02-01 00:00:01.
Built for Python 3.8 on linux (64-bit).



In [3]:
BASE_PATH = "/home/bernardo/Scaricati"
DOC_NAME = "Table_Detection_in_Invoice_Documents_by_Graph_Neural_Networks.pdf"
DOC_PATH = BASE_PATH + "/" + DOC_NAME
doc = fitz.open(f"{DOC_PATH}")
page = doc[1] # apri pure più pagine oltre la prima

In [4]:
from fitz.utils import getColorList
cl = getColorList()

color_line = fitz.utils.getColor("BLUE")
color_rect = fitz.utils.getColor("ORANGE")
color_bezr = fitz.utils.getColor("BROWN")
color_block = fitz.utils.getColor("RED")
color_textline = fitz.utils.getColor("PINK")
color_word = fitz.utils.getColor("YELLOW")
color_images = fitz.utils.getColor("GREEN")

In [5]:
# immagini
images = page.get_image_info()
images

[{'number': 6,
  'bbox': (313.5959167480469,
   70.18290710449219,
   552.2298583984375,
   224.615234375),
  'transform': (238.63392639160156,
   0.0,
   -0.0,
   154.4323272705078,
   313.5959167480469,
   70.18290710449219),
  'width': 1262,
  'height': 816,
  'colorspace': 3,
  'cs-name': 'ICCBased(RGB,sRGB IEC61966-2.1)',
  'xres': 96,
  'yres': 96,
  'bpc': 8,
  'size': 116675},
 {'number': 7,
  'bbox': (313.5959167480469,
   224.121337890625,
   552.2298583984375,
   255.42645263671875),
  'transform': (238.63392639160156,
   0.0,
   -0.0,
   31.30511474609375,
   313.5959167480469,
   224.121337890625),
  'width': 1262,
  'height': 165,
  'colorspace': 3,
  'cs-name': 'ICCBased(RGB,sRGB IEC61966-2.1)',
  'xres': 96,
  'yres': 96,
  'bpc': 8,
  'size': 18148}]

In [6]:
# linee, rettangoli etc.
draws = page.get_drawings()
for path in draws:
    for item in path["items"]:  # these are the draw commands
        # ------------------------------------
        # draw each entry of the 'items' list
        # ------------------------------------
        if item[0] == "l":  # line
            page.draw_line(item[1], item[2], color=color_line)
        elif item[0] == "re":  # rectangle
            page.draw_rect(item[1], color=color_rect)
        elif item[0] == "c":  # curve
            page.draw_bezier(item[1], item[2], item[3], item[4], color=color_bezr)
            
doc.save(f'{BASE_PATH}/page_draws.pdf')

In [7]:
# TESTO
texts = page.get_text("json")
texts = json.loads(texts)
for block in texts['blocks']:
    page.draw_rect(Rect(block['bbox']), color=color_block)
    if block['type'] == 0:      # all elements
        for line in block['lines']:
            page.draw_rect(Rect(line['bbox']), color=color_textline)
    elif block['type'] == 1:    # images
        page.draw_rect(Rect(block['bbox']), color=color_images)

doc.save(f'{BASE_PATH}/page_texts.pdf')

In [8]:
# parole
words = page.get_text('words')
for word in words:
    print(word)
    word_rect = Rect(word[0], word[1], word[2], word[3])
    page.draw_rect(word_rect, color=color_word)

doc.save(f'{BASE_PATH}/page_words.pdf')

(58.52629852294922, 70.64691925048828, 96.19466400146484, 82.41217803955078, 'discovery', 0, 0, 0)
(98.88944244384766, 70.64691925048828, 139.3013916015625, 82.41217803955078, 'algorithm.', 0, 0, 1)
(142.0059356689453, 70.64691925048828, 163.7008819580078, 82.41217803955078, 'Since', 0, 0, 2)
(166.39566040039062, 70.64691925048828, 186.46006774902344, 82.41217803955078, 'these', 0, 0, 3)
(189.15484619140625, 70.64691925048828, 208.13548278808594, 82.41217803955078, 'local', 0, 0, 4)
(210.83026123046875, 70.64691925048828, 248.79153442382812, 82.41217803955078, 'structures', 0, 0, 5)
(251.49607849121094, 70.64691925048828, 269.58819580078125, 82.41217803955078, 'have', 0, 0, 6)
(272.2829895019531, 70.64691925048828, 276.6180725097656, 82.41217803955078, 'a', 0, 0, 7)
(279.3128662109375, 70.64691925048828, 296.67266845703125, 82.41217803955078, 'high', 0, 0, 8)
(58.52629852294922, 82.36334991455078, 100.2270736694336, 94.12860870361328, 'variability,', 0, 1, 0)
(102.6875228881836, 82.363